In [1]:
# Goal 1

import csv
from collections import namedtuple


def get_dialect(fname):
    with open(fname, 'r') as f:
        return csv.Sniffer().sniff(f.read(1000))


class FileParser:
    def __init__(self, fname):
        self._fname = fname

    def __iter__(self):
        return self

    def __next__(self):
        if not self._file.closed:
            return self._namedtuple(*next(self._reader))
        raise StopIteration
    
    def __enter__(self):
        print('---')
        print(f'{self._fname} opening...')
        print('---')

        self._file = open(self._fname, 'r')
        self._reader = csv.reader(self._file, get_dialect(self._fname))
        header_row = map(lambda s: s.lower(), next(self._reader))
        self._namedtuple = namedtuple('Data', header_row)
        return self

    def __exit__(self, exc_type, exc_value, exc_trace_back):
        print('---')
        print(f'{self._fname} closing...')        
        print('---')

        self._file.close()
        return False


# Goal 2

import csv
from collections import namedtuple
from contextlib import contextmanager


@contextmanager
def parsed_data(fname):
    print('---')
    print(f'{fname} opening...')
    print('---')

    file = open(fname, 'r')
    try:
        dialect = csv.Sniffer().sniff(file.read(1000))
        file.seek(0)
        
        reader = csv.reader(file, dialect)
        header_row = map(lambda s: s.lower(), next(reader))
        nt = namedtuple('Data', header_row)
        yield (nt(*row) for row in reader)
    finally:
        print('---')
        print(f'{fname} closing...')
        print('---')

        file.close()
